In [4]:
import numpy as np
import pandas as pd

In [5]:
df=pd.read_csv('AMZN_stock_data.csv')
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1,1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
2,1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
3,1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
4,1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
...,...,...,...,...,...,...,...,...
6982,2025-02-14 00:00:00-05:00,229.199997,229.889999,227.229996,228.679993,27031100,0.0,0.0
6983,2025-02-18 00:00:00-05:00,228.820007,229.300003,223.720001,226.649994,42975100,0.0,0.0
6984,2025-02-19 00:00:00-05:00,225.520004,226.830002,223.710007,226.630005,28566700,0.0,0.0
6985,2025-02-20 00:00:00-05:00,224.779999,225.130005,221.809998,222.880005,30001700,0.0,0.0


In [6]:
df['Date']= pd.to_datetime(df['Date'])

/var/folders/n_/1hzw47553v1bvg9z8kct6rd40000gn/T/ipykernel_38993/2691600196.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date']= pd.to_datetime(df['Date'])


In [7]:
df['Close'].describe()

count    6987.000000
mean       40.685061
std        58.125169
min         0.069792
25%         2.110000
50%         9.021000
75%        73.006500
max       242.059998
Name: Close, dtype: float64

In [8]:
df['High'].describe()

count    6987.000000
mean       41.165619
std        58.787641
min         0.072396
25%         2.148250
50%         9.158500
75%        73.643497
max       242.520004
Name: High, dtype: float64

In [9]:
df['Close']=df['Close'].apply(lambda x: 1 if x>40.685061 else 0)
df['High']=df['High'].apply(lambda x: 1 if x>41.165619 else 0)


In [10]:
final=df[['Date','High','Close']]

In [11]:
final.set_index('Date')

,High,Close
Date,,
1997-05-15 00:00:00-04:00,0,0
1997-05-16 00:00:00-04:00,0,0
1997-05-19 00:00:00-04:00,0,0
1997-05-20 00:00:00-04:00,0,0
1997-05-21 00:00:00-04:00,0,0
...,...,...
2025-02-14 00:00:00-05:00,1,1
2025-02-18 00:00:00-05:00,1,1
2025-02-19 00:00:00-05:00,1,1


In [12]:
# Step 1: Build state and observation label lists
states = sorted(final['High'].unique())       # e.g. [0, 1]
observations = sorted(final['Close'].unique())# e.g. [0, 1]

In [23]:
def calculate_emission_matrix(observations_seq, states_seq, obs_symbols=None, state_symbols=None):
    if obs_symbols is None:
        obs_symbols = sorted(set(observations_seq))
    if state_symbols is None:
        state_symbols = sorted(set(states_seq))
    obs_idx = {obs: i for i, obs in enumerate(obs_symbols)}
    state_idx = {state: i for i, state in enumerate(state_symbols)}
    num_states = len(state_symbols)
    num_obs = len(obs_symbols)
    emission_matrix = np.zeros((num_states, num_obs))
    for s, o in zip(states_seq, observations_seq):
        i = state_idx[s]
        j = obs_idx[o]
        emission_matrix[i, j] += 1
    # Normalize rows
    row_sums = emission_matrix.sum(axis=1, keepdims=True)
    emission_matrix = np.divide(emission_matrix, row_sums, where=row_sums!=0)
    return emission_matrix, state_symbols, obs_symbols

emit_prob, state_labels, obs_labels = calculate_emission_matrix(
    final['Close'].values, final['High'].values, observations, states
)
emit_prob

array([[0.99858357, 0.00141643],
       [0.        , 1.        ]])

In [24]:
def calculate_transition_matrix(states_sequence, all_states=None, smoothing=0):
    if all_states is None:
        states = sorted(set(states_sequence))
    else:
        states = list(all_states)
    state_idx = {state:i for i, state in enumerate(states)}
    num_states = len(states)
    counts = np.zeros((num_states, num_states))
    for t in range(1, len(states_sequence)):
        i = state_idx[states_sequence[t-1]]
        j = state_idx[states_sequence[t]]
        counts[i, j] += 1
    if smoothing > 0:
        counts += smoothing
    row_sums = counts.sum(axis=1, keepdims=True)
    transition_matrix = np.divide(counts, row_sums, where=row_sums!=0)
    return transition_matrix, states

trans_prob, state_labels = calculate_transition_matrix(final['High'].values, states)
trans_prob

array([[9.99190611e-01, 8.09388911e-04],
       [1.46771037e-03, 9.98532290e-01]])

Above you see a df that we are gonna use. We will use it to make an HMM model where our observations is the Close column and the Hidden state is the High column

Goal: We are going to predict a hidden state(some Si) given some observations () i.e. Forward Algorithm = P(si=1|O1,...Ot)

In [15]:
"Your task: You guys need to calculate the Emission Matrix and Transition Matrix as well as the initial forward inference"

'Your task: You guys need to calculate the Emission Matrix and Transition Matrix as well as the initial forward inference'

In [16]:
class HiddenMarkovModel:
    """A Hidden markov model which takes Transition matrix and Emmission matrix as inputs"""

    def __init__(self, transition_matrix, emission_matrix, prior=None):
      self.transition_matrix = transition_matrix
      self.emission_matrix = emission_matrix
      self.prior = prior
      return None

    def emission_dist(self, ev):
      if ev:
        return self.emission_matrix[0]
      else:
        return self.emission_matrix[1]
      return None


In [17]:
def forward_step(trans, fv, ev):
  emission_dist = emission_dist(ev)
  forward_inf = fv
  transition_matrix = trans

  nonNormSolution = emission_dist * np.array([[1,0],[0,1]]) @transition_matrix @ forward_inf
  alpha = nonNormSolution / np.sum(nonNormSolution)


  return alpha

In [18]:
def forward_algorithm(Hmm,observations):
    T = len(observations)
    N = len(Hmm.piror)

    alpha = np.zeros((T,N))

    alpha[0] = Hmm.prior * Hmm.emission_dist(observations[0])
    alpha[0] = alpha[0] / np.sum(alpha[0])

    for t in range(1,T):
        alpha[t] = forward_step(Hmm, alpha[t-1],observations[t])
    return alpha

In [19]:
N = len(states)
start_prob = np.ones(N) / N  # Uniform; or use start_prob = [count of each state at 0]/total

In [20]:
# Step 5: Prepare the observation sequence (integer encoded)
obs_seq = [obs_labels.index(v) for v in final['Close'].values]
# (if obs_labels = [0, 1] and your Close is these values, this is [0, 1, 1, 0, ...])

In [21]:
# ---- Backward algorithm ----
def backward_algorithm(obs, states, start_prob, trans_prob, emit_prob):
    N = len(states)
    T = len(obs)
    backward = np.zeros((T, N))
    backward[T-1, :] = 1
    for t in range(T-2, -1, -1):
        for i in range(N):
            backward[t, i] = np.sum(trans_prob[i, :] * emit_prob[:, obs[t+1]] * backward[t+1, :])
    prob = np.sum(start_prob * emit_prob[:, obs[0]] * backward[0, :])
    return backward, prob

backward_matrix, sequence_prob = backward_algorithm(
    obs_seq, states, start_prob, trans_prob, emit_prob
)

print("Backward probability matrix shape:", backward_matrix.shape)
print("Probability of observation sequence under the HMM:", sequence_prob)

Backward probability matrix shape: (6987, 2)
Probability of observation sequence under the HMM: 2.5996747717102055e-36


In [22]:
backward_matrix, sequence_prob = backward_algorithm(
    obs_seq, states, start_prob, trans_prob, emit_prob
)

print("Backward probability matrix shape:", backward_matrix.shape)
print("Probability of observation sequence under the HMM:", sequence_prob)

Backward probability matrix shape: (6987, 2)
Probability of observation sequence under the HMM: 2.5996747717102055e-36
